In [1]:
from qiskit_emulator import EmulatorProvider

In [2]:
provider = EmulatorProvider()

In [3]:
provider.remote('http://qre-vf.oro-sandbox-small1.k8s.cec.lab.emc.com')

In [4]:
provider.runtime.programs()

{'4f2b45518fa1': <qiskit.providers.ibmq.runtime.runtime_program.RuntimeProgram at 0x7f42372abf70>,
 'cb9e10dcb3dd': <qiskit.providers.ibmq.runtime.runtime_program.RuntimeProgram at 0x7f42372abeb0>,
 'a63f5461c788': <qiskit.providers.ibmq.runtime.runtime_program.RuntimeProgram at 0x7f42372abe50>}

In [5]:
RUNTIME_PROGRAM = """
from qiskit.compiler import transpile, schedule


def main(
    backend,
    user_messenger,
    circuits,
    **kwargs,
):

    user_messenger.publish({'results': 'intermittently'})

    circuits = transpile(
        circuits,
    )

    if not isinstance(circuits, list):
        circuits = [circuits]

    # Compute raw results
    result = backend.run(circuits, **kwargs).result()

    user_messenger.publish({'results': 'finally'})
    user_messenger.publish(result.to_dict(), final=True)
    print("job complete successfully")
"""

In [6]:
RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

In [7]:
program_id = provider.runtime.upload_program(RUNTIME_PROGRAM, metadata=RUNTIME_PROGRAM_METADATA)

In [8]:
proglist = provider.runtime.programs()
proglist

{'4f2b45518fa1': <qiskit.providers.ibmq.runtime.runtime_program.RuntimeProgram at 0x7f42372abf70>,
 'cb9e10dcb3dd': <qiskit.providers.ibmq.runtime.runtime_program.RuntimeProgram at 0x7f42372abeb0>,
 'a63f5461c788': <qiskit.providers.ibmq.runtime.runtime_program.RuntimeProgram at 0x7f42372abe50>}

In [9]:
program_id

'ba720325586d'

In [10]:
from qiskit import QuantumCircuit, execute, transpile
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

program_inputs = {
    'circuits': qc,
}

In [11]:
job = provider.runtime.run(program_id, options=None, inputs=program_inputs)

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/geoff/miniconda3/envs/qruntime/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/home/geoff/miniconda3/envs/qruntime/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/home/geoff/miniconda3/envs/qruntime/lib/python3.9/site-packages/qiskit_emulator/emulator_runtime_job.py", line 118, in remote_poll_for_results
    while not self._finalResults and not self._kill and not self.job_completed():     
  File "/home/geoff/miniconda3/envs/qruntime/lib/python3.9/site-packages/qiskit_emulator/emulator_runtime_job.py", line 85, in job_completed
    self.status()
  File "/home/geoff/miniconda3/envs/qruntime/lib/python3.9/site-packages/qiskit_emulator/emulator_runtime_job.py", line 226, in status
    response.raise_for_status()
  File "/home/geoff/miniconda3/envs/qruntime/lib/python3.9/site-packages/requests/models.py", line 943, in raise_f

In [12]:
result = job.result(timeout=120)

TypeError: exceptions must derive from BaseException